In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
from requests import get

In [2]:
url = 'https://github.com/search?p=1&q=stars%3A%3E0&s=stars&type=Repositories'
headers  = {'User-Agent' : 'Codeup Data Science Student'}
response = get(url,headers)
soup = BeautifulSoup(response.text)
repo_list = soup.find('ul', class_='repo-list')
links = repo_list.find_all(href = True)
repo_links= [link['href'] for link in links if not re.match(r'^/topics',link['href']) and not re.match(r'^https',link['href'])\
         and link['href'].count('/') < 3]

In [3]:
repo_links

['/freeCodeCamp/freeCodeCamp',
 '/996icu/996.ICU',
 '/vuejs/vue',
 '/facebook/react',
 '/tensorflow/tensorflow',
 '/twbs/bootstrap',
 '/EbookFoundation/free-programming-books',
 '/sindresorhus/awesome',
 '/getify/You-Dont-Know-JS',
 '/ohmyzsh/ohmyzsh']

In [ ]:
pages = range(1,3)
pages